In [1]:
import numpy as np
import widgyts
import ipydatawidgets
import ipywidgets
import traitlets
from IPython.display import DisplayObject, display
import yt
import h5py

### Slice of Pop II prime dataset

This next cell demonstrates how the widget can be used to explore large parts of your dataset at once. By linking linking traitlets between widget views of different fields, you can interactively browse multiple fields at the same time. 

In [2]:
# reading in the projection data
with h5py.File("/Users/madicken/yt_data/scipydemo/DD0182_slice.h5", "r") as f:
    s = {}
    for field in ("px", "py", "pdx", "pdy", "density", "temperature"):
        s[field] = f[field][:]

# putting the fields into two different FRB views
frb_projd = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["density"], layout=ipywidgets.Layout(width = '500px', height='500px'))
frb_projt = widgyts.FRBViewer(height = 512, width = 512,
                            px = s["px"], py = s["py"], pdx = s["pdx"], pdy = s["pdy"],
                            val = s["temperature"], layout=ipywidgets.Layout(width = '500px', height='500px'))

# linking the views so their windows are aligned
ipywidgets.link((frb_projd, 'view_center'), (frb_projt, 'view_center'))
ipywidgets.link((frb_projd, 'view_width'), (frb_projt, 'view_width'))
ipywidgets.link((frb_projd.colormaps, 'is_log'), (frb_projt.colormaps, 'is_log'))

#adding in the log slider 
width_slider2 = ipywidgets.FloatLogSlider(value = 0.0, max = 0.0, min = np.log10(512 * s["pdx"].min()), base = 10)
width_slider2.value = 1.0
def on_changed(event):
    frb_projd.view_width = (event['new'], event['new'])
width_slider2.observe(on_changed, 'value', 'change')

#adding in the controls
controls = frb_projd.setup_controls()
display(ipywidgets.VBox([width_slider2,ipywidgets.HBox([controls,frb_projd, frb_projt])]))

# setting bounds of the temperature plot
frb_projt.colormaps.map_name='inferno'
frb_projt.colormaps.min_val=5.9
frb_projt.colormaps.max_val=15653

# setting the log scale
frb_projd.colormaps.is_log = True